In [1]:
import pinyin
import re
from collections import Counter

In [2]:
def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

In [3]:
def chinese_to_pinyin(character):
    ret = []
    character_clean = [''.join(token(str(a))) for a in character]
    for word in character_clean:
        ret += [pinyin.get(word,format='strip', delimiter=' ')]
    return ret

In [4]:
def get_token():
    ret = []
    with open( './article_9k.txt', 'r' ) as f:
        for line in f.readlines():
            ret += chinese_to_pinyin(line)
    return ret

In [39]:
def pinyin_counter(token):
    return Counter(token)

In [40]:
TOKEN = get_token()

In [41]:
TOKEN_2_GRAM = ['#'.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
TOKEN_2_GRAM[:10]

['ci#wai',
 'wai#zi',
 'zi#ben',
 'ben#zhou',
 'zhou#6',
 '6#yue',
 'yue#1',
 '1#2',
 '2#ri',
 'ri#qi']

In [42]:
words_count = pinyin_counter(TOKEN)
words_count.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('n', 683571),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441)]

In [43]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [44]:
def prob_1(word):
    return (words_count[word] + 1) / len(TOKEN)

In [45]:
def prob_2(word1, word2):
    if word1 + '#' + word2 in words_count_2: 
        return words_count_2[word1+'#'+word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

In [46]:
def split_words(spell):
    global split_sentence
    left_word = ''
    right_word = ''
    sentence = ''
    list_sentence = []
    prob = 0
    for i in range(2,len(spell)+1):
        left_word = spell[0:i]
        right_word = spell[i:]
        #print('left_word:{},right_word:{}'.format(left_word,right_word))
        
        if right_word == '':
            prob = prob_1(left_word)
            list_sentence += [(left_word, prob)]
            
        for i2 in range(2,len(right_word)+1):
            left_word2 = right_word[0:i2]
            right_word2 = right_word[i2:]
            sentence = left_word + ' ' + left_word2
            prob = prob_2(left_word, left_word2)
            list_sentence += [(sentence, prob)]
    #print(list_sentence)
    (select_sentence, select_pro) = max(list_sentence, key=lambda x: x[1])
    split_sentence += [(select_sentence, select_pro)]
    
    left_spell = spell[len(select_sentence)-1:]
    if left_spell != '' and spell != select_sentence:
        split_words(left_spell)

In [50]:
split_sentence = []
split_words('wokaibeijingtiananmen')

sentence = ''
for item in split_sentence:
    sentence += item[0] + ' '
print(sentence)

wo kai bei jing ti an an men 
